# Analyze Hurricane Paths
### Background
Using our cleaned HURDAT data, we will plot and compare the paths of hurricanes based on different time periods, to get a visual sense of how hurricanes have changed over time due to climate change.
### Contents
1. [Load Data](#1.-Load-Data)
2. [Compare Paths](#2.-Compare-Hurricane-Paths)

In [1]:
import os
import re
import folium
import pandas as pd
import numpy as np
import datetime
import pickle
from urllib.request import urlretrieve
from tqdm import tqdm_notebook as tqdm
from mpl_toolkits.basemap import Basemap

## 1. Load Data

In [2]:
with open('../data/hurricanes_cleaned.pkl', 'rb') as handle:
    hurricanes = pickle.load(handle)

In [3]:
hurricanes['AL062014']

{'name': 'EDOUARD',
 'datapoints': [{'record_date': '20140910',
   'time': '1800',
   'storm_status': 'LO',
   'lat': 13.7,
   'lon': -33.2,
   'max_wind': '25',
   'min_pressure': '1009',
   'category': 'TS',
   'dt': datetime.datetime(2014, 9, 10, 18, 0)},
  {'record_date': '20140911',
   'time': '0000',
   'storm_status': 'LO',
   'lat': 14.2,
   'lon': -34.4,
   'max_wind': '25',
   'min_pressure': '1009',
   'category': 'TS',
   'dt': datetime.datetime(2014, 9, 11, 0, 0)},
  {'record_date': '20140911',
   'time': '0600',
   'storm_status': 'LO',
   'lat': 14.9,
   'lon': -35.5,
   'max_wind': '25',
   'min_pressure': '1009',
   'category': 'TS',
   'dt': datetime.datetime(2014, 9, 11, 6, 0)},
  {'record_date': '20140911',
   'time': '1200',
   'storm_status': 'TD',
   'lat': 15.8,
   'lon': -36.5,
   'max_wind': '30',
   'min_pressure': '1008',
   'category': 'TS',
   'dt': datetime.datetime(2014, 9, 11, 12, 0)},
  {'record_date': '20140911',
   'time': '1800',
   'storm_status': 

## 2. Compare Hurricane Paths 
Before diving into tesitng our hypotheses, we want to look at some maps to visualize how hurricanes have changed over the last 100 years. We will start by looking at the periods 1918-1927 vs 2008-2017. 

Our hypothesis states that we expect:
1. More hurricanes in the 2008-2017 period
2. A higher number of very strong storms (Cat 4 & 5)

In [4]:
years_old = range(1918, 1928)
years_new = range(2008, 2018)

In [5]:
hurr_1918_1927 = [storm_data for storm_id, storm_data in hurricanes.items() if storm_data['year'] in years_old]
hurr_2008_2017 = [storm_data for storm_id, storm_data in hurricanes.items() if storm_data['year'] in years_new]

In [6]:
# Define mapping for colors depending on category
category_colors = {1: '#ffb3b3',
                   2: '#ff8080',
                   3: '#e60000',
                   4: '#ff0000',
                   5: '#580808',
                   'TS': '#ffe6e6'}

In [24]:
def create_path_map(storm_list, category_colors):
    # Initialize map
    hurr_map = folium.Map(location=[30, -60], zoom_start = 4)

    # Plot each storm's path with info in tooltip
    for storm in storm_list:
        storm_name = storm['name']

        for idx in range(len(storm['datapoints'])):
            point = storm['datapoints'][idx]

            # Handle last datapoint which will not have a future datapoint
            if idx+1 < len(storm['datapoints']):
                # Current point
                day_points = [(point['lat'], point['lon']), 
                              (storm['datapoints'][idx+1]['lat'], storm['datapoints'][idx+1]['lon'])]

                hurr_info = f"""
                Storm:        {storm_name}
                Date:         {point['record_date']}
                Time:         {point['time']}
                Status:       {point['storm_status']}
                Category:     {point['category']}
                Max Wind:     {point['max_wind']}mph
                Min Pressure: {point['min_pressure']}millibars
                """
                # Plot line
                folium.PolyLine(day_points, 
                                tooltip=storm_name, 
                                popup=hurr_info, 
                                color=category_colors[point['category']],
                               ).add_to(hurr_map)
                
    return hurr_map

In [25]:
map_1918_1927 = create_path_map(hurr_1918_1927, category_colors)
map_2008_2017 = create_path_map(hurr_2008_2017, category_colors)

In [26]:
# Save as html
map_1918_1927.save('../maps/hurricane_paths_1918-1927.html')
map_2008_2017.save('../maps/hurricane_paths_2008-2017.html')

In [15]:
# Initialize map
hurr_1900s_map = folium.Map(location=[30, -60],
                            zoom_start = 4)

# Plot each storm's path with info in tooltip
for storm in hurr_1918_1927:
    storm_name = storm['name']

    for idx in range(len(storm['datapoints'])):
        point = storm['datapoints'][idx]
        
        # Handle last datapoint which will not have a future datapoint
        if idx+1 < len(storm['datapoints']):
            # Current point
            day_points = [(point['lat'], point['lon']), 
                          (storm['datapoints'][idx+1]['lat'], storm['datapoints'][idx+1]['lon'])]

            hurr_info = f"""
            Storm:        {storm_name}
            Date:         {point['record_date']}
            Time:         {point['time']}
            Status:       {point['storm_status']}
            Category:     {point['category']}
            Max Wind:     {point['max_wind']}mph
            Min Pressure: {point['min_pressure']}millibars
            """
            # Plot line
            folium.PolyLine(day_points, 
                            tooltip=storm_name, 
                            popup=hurr_info, 
                            color=category_colors[point['category']],
                           ).add_to(hurr_1900s_map)

In [16]:
# Save as html
hurr_1900s_map.save('../maps/hurricane_paths_1918-1927_NEW.html')